In [59]:
import numpy as np
import pandas as pd
import scipy

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Methods implementation

### Loss & cost functions

In [60]:
def linear_prediction(w, X):
    """
    Returns a linear regression model prediction labels for objects in matrix 
    X using weights w:
    y_pred = (X,w)
    """
    if X.ndim == 1:
        return(np.insert(X, 0, 1).dot(w))
    else:
        n = X.shape[0]
        return np.dot(np.hstack((np.ones((n,1)),X)),w)
    
    
def mean_squared_error(y, y_pred):
    """
    Returns a mean squared error between real and predicted labels
    """
    y = np.array(y)
    y_pred = np.array(y_pred)
    mse = np.sum((y - y_pred)**2) / y.size
    return mse

def mean_absolute_error(y, y_pred):
    """
    Returns a mean absolute error between real and predicted labels
    """
    y = np.array(y)
    y_pred = np.array(y_pred)
    mae = np.sum(abs(y - y_pred)) / y.size
    return mae

def cost_function(w, X, y, type_f='mse'):
    """
    Returns a cost function of a linear model with coefficients w
    oh features X and labels y.
    """
    if (type_f == 'mae'):
        return mean_absolute_error(y, linear_prediction(w, X))
    elif (type_f == 'mse'):
        return mean_squared_error(y, linear_prediction(w, X))
    else:
        #raise
        print('error: incorrect type of function')
        return

### Linear regression model

In [61]:
def linear_regression_fit(X, y, minimize='grad_desc', cost_f='mse',
                          w0=None, eta=1e-2):
    """
    Returns weights that minimizes cost function.
    """   
    if (minimize == 'analytical'):
        # the cost function is 'mse' automatically
        # X_t = X.transpose()
        # w = np.linalg.inv(X_t.dot(X)).dot(X_t).dot(y)
        n = X.shape[0]
        X = np.hstack((np.ones((n,1)),X))
        w = np.linalg.lstsq(X,y)[0]
        
    elif (minimize == 'grad_desc'):
        # the cost function is 'mse' automatically
        if(w0 == None):
            w0 = np.zeros(X.shape[1] + 1)
        n = X.shape[0]
        X = np.hstack((np.ones((n,1)),X))
        w = gradient_descent(X, y, w0, eta)[0]
        w = w.reshape(w.shape[0])

    elif (minimize == 'st_grad_desc'):
        # the cost function is 'mse' automatically
        if(w0 == None):
            w0 = np.zeros(X.shape[1] + 1)
        n = X.shape[0]
        X = np.hstack((np.ones((n,1)),X))
        w = stochastic_gradient_descent(X, y, w0, eta)[0]
        w = w.reshape(w.shape[0])
    
    elif (minimize == 'scipy_minimize'):
        if(w0 == None):
            w0 = np.zeros(X.shape[1] + 1)
        w = scipy.optimize.minimize(lambda w: cost_function(w, X, y, cost_f), w0).x
    else:
        #raise
        print('error: incorrect minimization method')
        return
    return w

### Gradient descent

In [101]:
def gradient_step(X, y, w, eta=0.01):
    grad = 2 * X.T.dot(X.dot(w) - y) / X.shape[0] 
    w_next = w - eta * grad 
    return w_next

def gradient_descent(X, y, w0=None, eta=1e-2,
                     max_iter=1e4, min_weight_dist=1e-8):    
    X = np.array(X)
    y = np.array(y)
    if(w0 == None):
        w0 = np.zeros(X.shape[1])
        
    w = w0
    weight_dist = np.inf
    errors = []
    iter_num = 0
        
    while weight_dist > min_weight_dist and iter_num < max_iter: 
        w_next = gradient_step(X, y, w, eta)        
        errors.append(mean_squared_error(y, X.dot(w_next)))
        weight_dist = np.linalg.norm(w_next - w)      
        w = w_next
        iter_num += 1
    return w, errors

### Stochastic gradient descent

In [102]:
def stochastic_gradient_step(X, y, w, k, eta=0.01):
    grad = 2 * (X[k].dot(w) - y[k]) * X[k,:] / X.shape[0] 
    w_next = w - eta * grad   
    return w_next

def stochastic_gradient_descent(X, y, w0=None, eta=1e-2, max_iter=1e4,
                                min_weight_dist=1e-8, seed=42):    
    X = np.array(X)
    y = np.array(y)
    if(w0 == None):
        w0 = np.zeros(X.shape[1])
        
    w = w0
    weight_dist = np.inf
    errors = []
    iter_num = 0
    #import random
    #k = np.array(range(X.shape[0]))
    #random.shuffle(k)
    np.random.seed(seed)
        
    while weight_dist > min_weight_dist and iter_num < max_iter:  
        #print('\nit.',iter_num)
        #print('w: ',w)
        random_ind = np.random.randint(X.shape[0])
        #print('k: ',random_ind)
        w_next = stochastic_gradient_step(X, y, w, random_ind, eta)     
        errors.append(mean_squared_error(y, X.dot(w_next)))
        weight_dist = np.linalg.norm(w_next - w)      
        #print('d: ',weight_dist)
        w = w_next
        iter_num += 1
    return w, errors

# Data

### Generate / Import dataset

In [64]:
def normalize(X, mean_std=True):
    if mean_std:
        means, stds = X.mean(axis=0), X.std(axis=0)
        X = (X - means) / stds
    else:
        minim, maxim = data.min(axis = 0), data.max(axis = 0)
        X = (X - minim) / (maxim - minim)
    return X

In [80]:
from sklearn import datasets 

sample_size = 200
data, target = datasets.make_regression(n_samples = sample_size,
                                        n_features = 5, 
                                        n_informative = 4, 
                                        n_targets = 1, noise = 5.,
                                        coef = False, random_state = 2)
#data = normalize(data)

In [79]:
#data = pd.read_csv('Data/weights_heights.csv', index_col='Index')
data_frame = pd.read_csv('Data/advertising.csv')

features = ['TV'] # , 'Radio','Newspaper']
labels = ['Sales']
data = np.array(data_frame[features].values, dtype=float)[:sample_size]
target = np.array(data_frame[labels].values, dtype=float)[:sample_size]
target = target.reshape(target.shape[0])

data = normalize(data)

### Split dataset into train & test samples

In [81]:
from sklearn import cross_validation
train_data, test_data, train_labels, test_labels = \
cross_validation.train_test_split(data, target, test_size = 0.3)

In [82]:
print('train_data: \n',train_data[:5],'\n...\n')
print('train_labels: \n',train_labels[:5],'...')

train_data: 
 [[ 0.56266908  1.14485538 -0.15211769  0.82978905 -0.16451506]
 [-0.87810789  0.25657045 -0.98877905 -0.33882197 -0.15643417]
 [ 0.70201352  0.43198898 -1.01786273  0.28774717 -0.35613593]
 [-1.06492788  0.30752553 -1.36963867  2.67033724 -1.40084545]
 [ 1.42496703  0.52110943 -0.70819438  1.15553059 -0.4468579 ]] 
...

train_labels: 
 [ 118.35335449  -83.4536414    58.84383323 -184.41198763  133.15803617] ...


# Run models and output

### Auxiliary functions

In [83]:
def print_result(coef, true, predict, cut = 5):
    print('w:\n',coef,'\n')
    print('true vs. prediction:\n',vstack((true,predict)).T[:cut],'\n...')

In [84]:
def plot_for_one_feature(train_data, train_labels, w, title):
    x = np.linspace(train_data.min(), train_data.max(), 2).reshape((2,1))
    plt.figure(figsize = (8,5))
    plt.plot(train_data, train_labels, 'o', markersize = 3)
    #print(x)
    #print(linear_prediction(w, x))
    plt.plot(x, linear_prediction(w,x), '-', linewidth = 4)
    plt.xlabel('feature')
    plt.ylabel('label')
    plt.title(title)
    plt.show()

### Analytical OLS method

In [85]:
w = linear_regression_fit(train_data, train_labels, minimize='analytical')
print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ -0.36559549  91.99673611  71.70021632  11.91772276   0.611216    73.8495751 ] 

true vs. prediction:
 [[  -2.99954166    1.39857714]
 [ 151.99687856  148.36137291]
 [ -21.3469044   -14.77390173]
 [ 210.32076737  209.62970353]
 [   9.77306619   14.58461328]] 
...


### Numerical method using gradient descent

In [87]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels,
                          minimize='grad_desc', eta=0.1)

print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ -0.36559551  91.99673611  71.70021629  11.91772272   0.61121599
  73.84957509] 

true vs. prediction:
 [[  -2.99954166    1.39857717]
 [ 151.99687856  148.36137291]
 [ -21.3469044   -14.7739017 ]
 [ 210.32076737  209.62970353]
 [   9.77306619   14.58461335]] 
...


/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  # This is added back by InteractiveShellApp.init_path()


### Numerical method using stochastic gradient descent

In [100]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels,
                          minimize='st_grad_desc', eta=0.05)

print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  # This is added back by InteractiveShellApp.init_path()


w:
 [ -0.31638139  91.75245422  71.40564143  11.67582924   0.49480574
  73.72921677] 

true vs. prediction:
 [[  -2.99954166    1.74852984]
 [ 151.99687856  148.27061492]
 [ -21.3469044   -14.31318066]
 [ 210.32076737  209.46375159]
 [   9.77306619   14.98328227]] 
...


### Numerical method using scipy.optimize.minimize

In [89]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels, minimize='scipy_minimize', cost_f='mae')

w.reshape((1,w.shape[0]))
print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ -0.60850538  91.53711828  71.42394879  12.31444398   0.35759316
  73.47782498] 

true vs. prediction:
 [[  -2.99954166    0.97157278]
 [ 151.99687856  147.25743655]
 [ -21.3469044   -15.28496548]
 [ 210.32076737  207.68950867]
 [   9.77306619   13.15112739]] 
...


### sklearn.linear_model.SGDRegression for check

In [98]:
from sklearn import linear_model 

sgd_regressor = linear_model.SGDRegressor(random_state=None, n_iter=20)
sgd_regressor.fit(train_data, train_labels)
sgd_regressor.predict(test_data)
w = [sgd_regressor.intercept_[0]]
w.extend(sgd_regressor.coef_)
w = np.array(w)
print_result(w,test_labels,sgd_regressor.predict(test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ -0.99235873  91.1684989   71.02926226  10.599685     0.23201854
  73.214544  ] 

true vs. prediction:
 [[  -2.99954166    1.69176417]
 [ 151.99687856  147.53276654]
 [ -21.3469044   -14.0181392 ]
 [ 210.32076737  208.33176954]
 [   9.77306619   16.10693402]] 
...
